In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest, f_classif, mutual_info_classif, RFE, SelectFromModel
from sklearn.ensemble import RandomForestClassifier
import warnings
warnings.filterwarnings("ignore")

pd.options.display.max_columns = None
pd.options.mode.chained_assignment = None  # default='warn'
plt.rcParams['axes.grid'] = True

### Data Anlaysis :
##### Main aim is to understand more about the data.
- Categorical Variables: 
    - How many.
    - Cardinality of Categorical Variables (bar): to decide later which to use label encoding or one-hot encoding
    
- Numerical Variables:
   - How many.
   - Distribution of the Numerical Variables (histogram,line): use this information later at the preoprocessing stage.
      - Data skewed left\right => data has outliers or has many missing values.
      - The goal is to convert non noraml distribution to more like normal one.
      - The converting is done by: cleaning data, removing outliers, filling missing values with median if outliers exist, etc.

- All Features:
    - Missing Values: calculate NAN% in each var to decide whether to drop the feature or filling it with the proper strategy
    - Duplicates
    - Outliers: detect Outliers with(boxplot, IQR) 
    - Relationship between independent and dependent feature(machine_status): gonna postpone this step till later after label encoding the target and setting timestamp as our data index

In [ ]:
dataset = pd.read_csv("/kaggle/input/pump-sensor-data/sensor.csv")
dataset.shape

In [ ]:
dataset.head(5)

In [ ]:
dataset.dtypes

In [ ]:
#We'll have to convert timestamp to datetime and maybe index our data on it
#also we may have to categories the target 'machine_status'

### Categorical variables:
- How many?
- Cardinality of each feature: to decide later which to use label encoding or one-hot encoding (use bar chart)

In [ ]:
dataset.describe(include='O')

count only counts non-null values.
timestamp: count = unique ==> no nulls + no duplicate ==> use timestamp as our data index later at the data preprocessing stage

In [ ]:
sns.countplot(x=dataset['machine_status'])

In [ ]:
#List machine_status distinct values:
dataset['machine_status'].unique().tolist()

In [ ]:
#check frequency of each value
dataset['machine_status'].value_counts()

machine_status: 3 distinct values, no nulls ==> use label-encoding since we only have 3 distict values

### Thoughts:

The water pumb got broken 7 times, which is a huge number for just one year.

We'll try to find out if there is any pattern that connects failure status to timestamp, does is it occur more at night time? early morning? afternoon? etc.

Of course if we have got more data or if sensors were defined appropriately instead of labelling them just with sensor number, we would have a more clear vision of what's going on. but for now we'll do our best with the data we have as it is.

In [ ]:
#get timestamp where status == broken
timestamp = dataset[['timestamp','machine_status']].loc[dataset['machine_status'].str.lower()=='broken']
timestamp.set_index('timestamp', inplace= True)
timestamp

### Thoughts:

All failues had happened twice almost for every month starting from April till July. except for june there was only one failure at the end of month. Could failure be releated to tempertaure rising during these months? The dataset owner is originated from Bangkok, so it's a safe guess to say the data is originated from Thailand. Summer in Thai starts from March to June, April being the hottest month in the year. The failures that occurred in July can be interpreted as the result of accumulations from June (assuming the stability of the electric current, it is logical to assume that the failure did not happen overnight, rather it was a gradual cumulative process). 85% of failures happenned between 10pm and 1am. These findings may have a meaning when presented to data owner.

### Numerical variables:
- Numerical Variables:
   - How many.
   - Distribution of the Numerical Variables (histogram,line): use this information later at the preoprocessing stage.
      - Data skewed left\right => data has outliers or has many missing values.
      - The goal is to convert non noraml distribution to more like normal one.
      - The converting is done by: cleaning data, removing outliers, filling missing values with median if outliers exist, etc.

In [ ]:
dataset.describe().transpose()

We have 52 continuous numerical features.
we suspect that the Unnamed: 0 and sensor_15 are both meaningless, the first is for numbering, the second is empty. 
we'll further investigate both.

In [ ]:
print('duplicates= %d' % dataset['Unnamed: 0'].duplicated().sum())
dataset['Unnamed: 0'].describe()

No Duplicates, min=0, max= 220319 ==> Obviously it's used as an index. We'll replace it later with timestamp.

In [ ]:
print('duplicates= %d' % dataset['sensor_15'].duplicated().sum())
dataset['sensor_15'].describe()

Safe to drop sensor_15 since it's null feature
### Data Distribution:

In [ ]:
##Group numerical features in one temporary dataframe for further analysis:
Numerical_features = dataset.select_dtypes(exclude=['object'])
# Since sensor data are continues values, we use histogram to understand the distribution
for feature in Numerical_features:
    data=dataset.copy()
    data[feature].hist(bins=25)
    plt.xlabel(feature)
    plt.ylabel("Count")
    plt.title(feature)
    plt.show()

We can see that we have the three distributions; i.e., normal, skewed left\right.
The distribution should be semi\normal after handling missing data and outliers, which we'll do at the preprocessing stage.
We also see that some features have alomst the same distribution. one can attribute this to the fact that in actual operation, pump systems are equipped with multi sensors for the same operating parameter, such as pressure, temperature, etc. for a couple of reasons such as system reliability, safety and so on. 
This also can cause measured signals' overlapping as seen next:

In [ ]:
dataset.plot(subplots =True, sharex = True, figsize = (20,50))

In [ ]:
#Try to detect trends in sensors when the machine gets broken
#This will help us in feature selection later
df = dataset.drop(['Unnamed: 0','sensor_15'], axis=1)
df['timestamp'] = pd.to_datetime(df['timestamp'])
df.set_index('timestamp', inplace= True)

df_broken = df[df['machine_status']=='BROKEN']
df_sensors = df.drop(['machine_status'], axis=1)
sensors = df_sensors.columns

for sensor in sensors:
    sns.set_context('paper')
    _ = plt.figure(figsize=(16,2))
    _ = plt.plot(df[sensor], color='green')
    _ = plt.plot(df_broken[sensor], linestyle='none', marker='X', color='red', markersize=8)   
    _ = plt.title(sensor)
    plt.show()

It seems that sensors <b>{0,4,5,6,7,10,11,12,13}</b> correlate a lot with the failure of the machine and can be a good indicator of the failure of the system. We'll keep this in mind when we select features later.

In [ ]:
#Plot the whole raw dataset to detect correlation between sensors and target, also multicollinearity if any. 
#This information also will help us in feature selecting later.
#label encode machine status to be able to plot it along with the other numeric sensors

df = dataset.drop(['Unnamed: 0','sensor_15'], axis=1)
df['timestamp'] = pd.to_datetime(df['timestamp'])
df.set_index('timestamp', inplace= True)

df['machine_status'] = df['machine_status'].map({'NORMAL': 0, 'BROKEN': 1, 'RECOVERING':1})

#df.dtypes
corr = df.corr()
plt.figure(figsize=(24,24))
sns.heatmap(corr, annot=True, fmt = '.1f')


- There is a strong correlation exists between sensor_14 to sensor_26, sensor_28 to sensor_33 and sensor_34 to sensor_36.
- Multicollinearity should be handled, otherwise ML model performance will decrease.
- machine_status is highly negative correlated with sensor_01 to sensor_12.

### All Features:
- Missing Values: calculate NAN% in each var to decide whether to drop the feature or filling it with the proper strategy
- Duplicates
- Outliers: detect Outliers with(boxplot, IQR) 
- Correlation between independent and dependent feature(machine_status)

### Finding Missing Values (NAN %):

In [ ]:
#As we've seen above, we only have two categorical features; machine_status and timestamp.
#Neither one of them has missing values. so here we'll concentrate only on numerical variables.
#Get Null Precentage in each column. I think % is more readable, but the plain view is fine as well.

features_with_na=[features for features in Numerical_features.columns if Numerical_features[features].isna().sum()>1]
na_percent = round(Numerical_features[features_with_na].isna().sum() * 100 / len(Numerical_features),2)
missing_percent_df = pd.DataFrame({'Feature': features_with_na, 'Na%':na_percent})

#Sort based on null precentage in each feature
missing_percent_df.sort_values('Na%', inplace=True, ascending=False)
missing_percent_df

### Finding Duplicates:

In [ ]:
#look for duplicated rows based on timestamp column
#false == all rows are unique rows == no duplicate rows
boolean = not dataset["timestamp"].is_unique
boolean

### Finding Outliers:

In [ ]:
#since so many of our features have skewed distribution, we'll use IQR method to detect outliers in these features:
for feature in Numerical_features:  
    series = Numerical_features[feature]
    Q1 = series.quantile(0.25) 
    Q3 = series.quantile(0.75) 
    IQR = Q3 - Q1
    lower = Q1 - (1.5 * IQR)
    upper = Q3 + (1.5 * IQR)
    outliers = [x for x in series if x < lower or x > upper]
    print('Identified outliers: %d' % len(outliers))
    outliers_removed = [x for x in series if x >= lower and x <= upper]
    print('Non-outlier observations: %d' % len(outliers_removed))
    print('Outliers in',feature,':', round(len(outliers) * 100 / len(series),2),'%')
    plt.figure(figsize=(6, 3))
    sns.boxplot(x= series)
    plt.show()

### Thoughts:

We know that our numeric features are sensors data, and we expected there will be outliers due to the fact that these sensors are working in a real enviroment and it is inevitable that some sensor nodes malfunction, which may result in noisy, faulty, missing and redundant data. Also the low cost and low quality sensor nodes have firm resource constraints such as energy (battery power), memory, computational capacity and communication bandwidth. The limited resource and capability affect the reliablity and the accuracy of the generated data. Especially when battery power is exhausted, the probability of generating erroneous data will grow rapidly. 

Even With that in mind, we're not sure if what appear as outliers in the plots are really outliers.
Could they be just part of the data? 

### Data Preprocessing
- Drop obvious null or irrelevant features
- Re-index dataset if needed
- Encode Label data: one-hot or label-encoding (in our case we choose label-enoding)
- Data Imputation: 
    - if numerical => constant Or statistics.
    - if categorical => constant or most-frequent.
    - if data has outliers, then median should be used as both mean and std. are affected greatly with the outliers.
- Handle Outliers by (we chose capping technique as we don't want to just trim them) 
- Data Scaling: we chose Robust Scaler just in case we still have outliers
- Remove Duplicates.
- Dataset Correlation
- Correlation between independent and target feature(machine_status): use this information later in feature selection

In [ ]:
#drop sensor_15, Unnamed: 0
dataset.drop({'sensor_15','Unnamed: 0'},axis=1, inplace=True)

#convert timestamp to datetime 
dataset['timestamp'] = pd.to_datetime(dataset['timestamp'])

#set timestamp as data index
dataset.set_index('timestamp', inplace= True)

### Label Encoding:

In [ ]:
#Replace the categorical value with a numeric value. 
#NORMAL is mapped to 0, RECOVERING and BROKEN are mapped to 1.
#We'll ue mapping here instead of LabelEncoder

dataset['machine_status'] = dataset['machine_status'].map({'NORMAL': 0, 'BROKEN': 1, 'RECOVERING':1})

In [ ]:
dataset['machine_status'].value_counts()

### Data Imputation
From plotting the features early in the EDA stage, clearly many of our data are skewed, plus there are many outlies. therefore we will use median value to fill in the missing values:

In [ ]:
#Filling missing numerical values with median
#first drop sensor_15 from features_with_na list
features_with_na.remove('sensor_15')
median_series = dataset[features_with_na].median()
dataset[features_with_na] = dataset[features_with_na].fillna(median_series)

In [ ]:
#Double check the whole dataframe again
na_values = dataset[features_with_na].isna().sum()
not all(na_values) #If true ==> no missing values anymore.

### Handle Outliers (Via Capping):

In [ ]:
#1. Cap ouliers in all features
for feature in dataset[features_with_na]:  
    percentiles = dataset[feature].quantile([0.25, 0.75]).values
    dataset[feature][dataset[feature] <= percentiles[0]] = percentiles[0]
    dataset[feature][dataset[feature] >= percentiles[1]] = percentiles[1]

In [ ]:
#2. plot features after capping to see the effect
for feature in dataset[features_with_na]:    
    Q1 = dataset[feature].quantile(0.25) 
    Q3 = dataset[feature].quantile(0.75) 
    IQR = Q3 - Q1
    lower = Q1 - (1.5 * IQR)
    upper = Q3 + (1.5 * IQR)
    outliers = [y for y in dataset[feature] if y < lower or y > upper]
    print('Outliers in',feature,':', round(len(outliers) * 100 / len(dataset[feature]),2),'%')
    plt.figure(figsize=(6, 3))
    sns.boxplot(x= dataset[feature])
    plt.show()

### Data Scaling:
Our features varies largly in their scales, therefore scaling them is a neccessity to prepare our data to be fed to ML model later.
We'll use Robust Scaler.

In [ ]:
scaler = preprocessing.RobustScaler()
dataset[features_with_na] = scaler.fit_transform(dataset[features_with_na])
dataset[features_with_na].describe()

### Remove Duplicates:

In [ ]:
dataset.drop_duplicates()

### Correlation:

1. Dataframe Correlation:
- This knowledge help us to prepare our data to meet the expectations of ML algorithms, such as classification (which we're going to use here) whose performance will degrade with the presence of these interdependencies (multicollinearity).
    
    +1 ==> high positive correlation
    
    -1 ==> high negative correlation

In [ ]:
corr = dataset.corr(method='spearman')
plt.figure(figsize=(24,24))
sns.heatmap(corr, annot=True, fmt = '.1f')

- Notice how scaling and outliers capping has affected the overall correlation.
- Instead of the strong positive correlation between sensor_14 to sensor_26 that we saw when data was raw, now the correlation is average.
- Same case with sensor_34 to sensor_36, sensor_22 to sensor_23 data.
- machine_status now has a slightly negative correlation with features from sensor_01 to sensor_12.

2. Correlation between machine_status and independent features:

In [ ]:
#Spearmanr is being used since most of our features don't follow a normal distribution
corr_matrix = dataset.corr(method='spearman')
print(corr_matrix["machine_status"].sort_values(ascending=False))

<b>Group</b> features data based on machine_status mean and plot the result to get a better sense of feature \ target relation 

In [ ]:
for feature in dataset.columns:
    dataset.groupby(feature)['machine_status'].mean().plot(legend=True)
    plt.ylabel('machine_status')
    plt.title(feature)
    plt.show()

### Feature Selection
Before applying feature selection method, we need to split the data first. The reason is that we only select features based on the information from the training set, not on the whole data set. We should hold out part of the whole data set as a test set to evaluate the performance of the feature selection and the model. Thus the information from the test set cannot be seen while we conduct feature selection and train the model.

In [ ]:
#Capture the dependent feature
y = dataset['machine_status']

#Drop dependent feature from dataset
X = dataset.drop('machine_status',axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=100, test_size=0.3)
#We will apply the feature selection based on X_train and y_train

We have numerical features as input and categorical feature as an output.
With that in mind we'll test 4 selection methods and see what comes out!

### 1. ANOVA: f_classif()

In [ ]:
# Univariate feature selection with F-test
# computes ANOVA f-value
# We select the most significant features
sel_f = SelectKBest(f_classif)
X_train_f = sel_f.fit_transform(X_train, y_train)
print(sel_f.get_support())

In [ ]:
feat_1 = X_train.columns[(sel_f.get_support())]
print('selected features: {}'.format(len(feat_1)))
feat_1

### 2. Mutual Information: mutual_info_classif()

In [ ]:
# Computes the mutual information
# Mutual information between two random variables is a non-negative value, which measures the dependency between the variables. 
# It is equal to zero if and only if two random variables are independent, and higher values mean higher dependency.

sel_mutual = SelectKBest(mutual_info_classif)
X_train_mutual = sel_mutual.fit_transform(X_train, y_train)
print(sel_mutual.get_support())

In [ ]:
feat_2 = X_train.columns[(sel_mutual.get_support())]
print('selected features: {}'.format(len(feat_2)))
feat_2

### 3. Recursive feature elimination (RFE): 
#### - Random forest as the model

In [ ]:
# Select features by recursively considering smaller and smaller sets of features.
# First, the estimator is trained on the initial set of features and the importance of each feature is obtained either 
# through a coef_ attribute or through a feature_importances_ attribute.
# Then, the least important features are pruned from the current set of features. 
# That procedure is recursively repeated on the pruned set until the desired number of features to select is eventually reached.

model_tree = RandomForestClassifier(random_state=100, n_estimators=50)
sel_rfe_tree = RFE(estimator=model_tree, step=1)
X_train_rfe_tree = sel_rfe_tree.fit_transform(X_train, y_train)
print(sel_rfe_tree.get_support())

In [ ]:
feat_3 = X_train.columns[(sel_rfe_tree.get_support())]
print('selected features: {}'.format(len(feat_3)))
feat_3

### 4. Select From Model: SelectFromModel
#### - Tree-based feature selection

In [ ]:
# The features are considered unimportant and removed if the corresponding coef_ or feature_importances_ values are 
# below the provided threshold parameter.
# Compared to univariate feature selection, model-based feature selection consider all feature at once,
# thus can capture interactions. 
# The model used for the feature selection doesn’t need to be the same model for the training later.

model_tree = RandomForestClassifier(random_state=100, n_estimators=50)
model_tree.fit(X_train, y_train)
print(model_tree.feature_importances_)
sel_model_tree = SelectFromModel(estimator=model_tree, prefit=True, threshold='mean')  
      # since we already fit the data, we specify prefit option here
      # Features whose importance is greater or equal to the threshold are kept while the others are discarded.
X_train_sfm_tree = sel_model_tree.transform(X_train)
print(sel_model_tree.get_support())

In [ ]:
# make a list of the selected features
feat_4 = X_train.columns[(sel_model_tree.get_support())]
print('selected features: {}'.format(len(feat_4)))
feat_4

Finally, we select features based on intersection between:
- Correlation analysis
- Anova: F-test
- Mutual_info_classif test
- Recursive feature elimination with random forest as the model
- SelectFromModel: tree based feature selection

In [ ]:
# Gonna intersect our four lists
list1_as_set = set(feat_1)

intersect = list1_as_set.intersection(feat_2) #1,2
intersect1 = intersect.intersection(feat_3) #2,3
intersect2 = intersect1.intersection(feat_4) #3,4

intersection_as_list = list(intersect2)

intersection_as_list.sort()
selected_features = intersection_as_list
selected_features
#0,4,5,6,7,10,11,12,13

In [ ]:
X_train = X_train[selected_features]
X_train.columns

### Model Building:

We'll try multiple models and select the most accurate one. 
Models include: Logistic Regression - Decision Tree - Random Forest - Xgboost

- Hyperparameter Tuning: it's required to get the most out of your Ml model. Random Search is used here.
- tol, max_iter are considered as parameters and its values are tol=[0.0001,0.001,0.01,0.1], max_iter=[20,50,100,200]
- tol represents tolerance for stopping criteria
- max_iter represents maximum number of iterations taken for solvers to converge
- Random search is the best parameter technique when there are less number of dimensions
- Evaluation with Confusion Matrix + Classification Report
- Model With Best Parameter
- Apply the selected best parameters to Logistic Regression and fit the model
- Predict the probability score using predict_proba() for train and test data
- Plot the roc curve for train and test data


### I. Logistic Regression

In [ ]:
from scipy.stats import loguniform
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import RandomizedSearchCV

# define model
model = LogisticRegression()
# define evaluation
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
# define search space
space = dict() #solver. Use either ‘liblinear’ or ‘saga
space['solver'] = ['liblinear']
space['penalty'] = ['l1']
space['C'] = loguniform(1e-5, 100)
# define search
search = RandomizedSearchCV(model, space, n_iter=500, scoring='accuracy', n_jobs=-1, cv=cv, random_state=1)

In [ ]:
# execute search
result = search.fit(X_train, y_train)

In [ ]:
# summarize result
print('Best Score: %s' % result.best_score_)
print('Best Hyperparameters: %s' % result.best_params_)

#### II. Decision Tree

#### III. Random Forest

#### IV. Xgboost